In [1]:
import os
import requests 
import time
import string
import networkx as nx
import itertools
import networkx as nx
import pandas as pd
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

#nltk.download('stopwords')

In [169]:
tweets_df =pd.read_csv("C:/Users/korab/Downloads/covid-tweets-master (2)/covid-tweets-master/data/4.countries/China.csv")

In [170]:
print(tweets_df)

                          created_at            id        id_str  \
0     Fri Feb 21 23:38:56 +0000 2020  1.230000e+18  1.230000e+18   
1     Fri Feb 21 23:20:00 +0000 2020  1.230000e+18  1.230000e+18   
...                              ...           ...           ...   
9348  Wed Mar 17 06:55:35 +0000 2021  1.370000e+18  1.370000e+18   
9349  Wed Mar 17 05:56:40 +0000 2021  1.370000e+18  1.370000e+18   

                                                                                                                                                  text  \
0       CGTN talked to W. Ian Lipkin of Columbia University on the challenges China is facing as it deals with misconceptioâ€¦ https://t.co/HltCFSmCsu   
1     Spring always brings along a lively vibe saturated by seas of colorful blooming flowers. \n\nTake a look at this collâ€¦ https://t.co/7tDC97NVaj   
...                                                                                                                              

In [171]:
tweets_filtered = tweets_df.copy() #it's a good idea to work on the copy of original dataframe, so we can always go back to it if we mess something up
column_list = ["created_at", "id_str", "text", "user", "retweeted_status", "quote_count", "reply_count", "retweet_count", "favorite_count", "entities"]
tweets_filtered = tweets_filtered[column_list]

In [172]:
tweets_filtered

,created_at,id_str,text,user,retweeted_status,quote_count,reply_count,retweet_count,favorite_count,entities
0,Fri Feb 21 23:38:56 +0000 2020,1.230000e+18,CGTN talked to W. Ian Lipkin of Columbia University on the challenges China is facing as it deals with misconceptioâ€¦ https://t.co/HltCFSmCsu,"{'id': 1115874631, 'id_str': '1115874631', 'name': 'CGTN', 'screen_name': 'CGTNOfficial', 'location': 'Beijing, China', 'url': 'http://www.CGTN.com', 'description': 'CGTN is an international media organization. It aims to provide global audiences with accurate and timely news coverage as well as...",NaN,0,1,7,24,"{'hashtags': [], 'urls': [{'url': 'https://t.co/HltCFSmCsu', 'expanded_url': 'https://twitter.com/i/web/status/1231000383151116289', 'display_url': 'twitter.com/i/web/status/1â€¦', 'indices': [117, 140]}], 'user_mentions': [], 'symbols': []}"
1,Fri Feb 21 23:20:00 +0000 2020,1.230000e+18,Spring always brings along a lively vibe saturated by seas of colorful blooming flowers. \n\nTake a look at this collâ€¦ https://t.co/7tDC97NVaj,"{'id': 1115874631, 'id_str': '1115874631', 'name': 'CGTN', 'screen_name': 'CGTNOfficial', 'location': 'Beijing, China', 'url': 'http://www.CGTN.com', 'description': 'CGTN is an international media organization. It aims to provide global audiences with accurate and timely news coverage as well as...",NaN,1,2,30,111,"{'hashtags': [], 'urls': [{'url': 'https://t.co/7tDC97NVaj', 'expanded_url': 'https://twitter.com/i/web/status/1230995615900934144', 'display_url': 'twitter.com/i/web/status/1â€¦', 'indices': [117, 140]}], 'user_mentions': [], 'symbols': []}"
...,...,...,...,...,...,...,...,...,...,...
9348,Wed Mar 17 06:55:35 +0000 2021,1.370000e+18,"LIVE: Pakistan begins vaccinating its citizens aged above 60 against COVID-19, with the help of Chinese-donated Sinâ€¦ https://t.co/Wn7hOtBgTb","{'id': 487118986, 'id_str': '487118986', 'name': 'China Xinhua News', 'screen_name': 'XHNews', 'location': 'Headquartered in Beijing, PRC', 'url': None, 'description': ""We are public media for the public good. We don't pursue corporate interests, nor will we yield to the pressure of ideological ...",NaN,0,2,10,45,"{'hashtags': [], 'urls': [{'url': 'https://t.co/Wn7hOtBgTb', 'expanded_url': 'https://twitter.com/i/web/status/1372079148584169481', 'display_url': 'twitter.com/i/web/status/1â€¦', 'indices': [117, 140]}], 'user_mentions': [], 'symbols': []}"
9349,Wed Mar 17 05:56:40 +0000 2021,1.370000e+18,#HeadlinesWatch: SCMP https://t.co/J15SRgQ5HM\n- 10 countries have administered 75% of all vaccinations worldwide whâ€¦ https://t.co/P6mb6xIjaZ,"{'id': 487118986, 'id_str': '487118986', 'name': 'China Xinhua News', 'screen_name': 'XHNews', 'location': 'Headquartered in Beijing, PRC', 'url': None, 'description': ""We are public media for the public good. We don't pursue corporate interests, nor will we yield to the pressure of ideological ...",NaN,5,1,16,28,"{'hashtags': [{'text': 'HeadlinesWatch', 'indices': [0, 15]}], 'urls': [{'url': 'https://t.co/J15SRgQ5HM', 'expanded_url': 'http://bit.ly/3vxPWSK', 'display_url': 'bit.ly/3vxPWSK', 'indices': [22, 45]}, {'url': 'https://t.co/P6mb6xIjaZ', 'expanded_url': 'https://twitter.com/i/web/status/13720643..."


In [173]:
import re
import emoji
import string

In [174]:
tweets_filtered['text'].iloc[100]

"China's home prices stay generally stable in January https://t.co/Og3StUcsYx"

In [175]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase
    table = str.maketrans(dict.fromkeys(string.punctuation)) 
    tweet = tweet.translate(table)# remove punctuation         
    return tweet

In [176]:
tweets_filtered["clean_text"] = tweets_filtered["text"].map(cleaner)
tweets_filtered

,created_at,id_str,text,user,retweeted_status,quote_count,reply_count,retweet_count,favorite_count,entities,clean_text
0,Fri Feb 21 23:38:56 +0000 2020,1.230000e+18,CGTN talked to W. Ian Lipkin of Columbia University on the challenges China is facing as it deals with misconceptioâ€¦ https://t.co/HltCFSmCsu,"{'id': 1115874631, 'id_str': '1115874631', 'name': 'CGTN', 'screen_name': 'CGTNOfficial', 'location': 'Beijing, China', 'url': 'http://www.CGTN.com', 'description': 'CGTN is an international media organization. It aims to provide global audiences with accurate and timely news coverage as well as...",NaN,0,1,7,24,"{'hashtags': [], 'urls': [{'url': 'https://t.co/HltCFSmCsu', 'expanded_url': 'https://twitter.com/i/web/status/1231000383151116289', 'display_url': 'twitter.com/i/web/status/1â€¦', 'indices': [117, 140]}], 'user_mentions': [], 'symbols': []}",cgtn talked to w ian lipkin of columbia university on the challenges china is facing as it deals with misconceptioâ€¦
1,Fri Feb 21 23:20:00 +0000 2020,1.230000e+18,Spring always brings along a lively vibe saturated by seas of colorful blooming flowers. \n\nTake a look at this collâ€¦ https://t.co/7tDC97NVaj,"{'id': 1115874631, 'id_str': '1115874631', 'name': 'CGTN', 'screen_name': 'CGTNOfficial', 'location': 'Beijing, China', 'url': 'http://www.CGTN.com', 'description': 'CGTN is an international media organization. It aims to provide global audiences with accurate and timely news coverage as well as...",NaN,1,2,30,111,"{'hashtags': [], 'urls': [{'url': 'https://t.co/7tDC97NVaj', 'expanded_url': 'https://twitter.com/i/web/status/1230995615900934144', 'display_url': 'twitter.com/i/web/status/1â€¦', 'indices': [117, 140]}], 'user_mentions': [], 'symbols': []}",spring always brings along a lively vibe saturated by seas of colorful blooming flowers take a look at this collâ€¦
...,...,...,...,...,...,...,...,...,...,...,...
9348,Wed Mar 17 06:55:35 +0000 2021,1.370000e+18,"LIVE: Pakistan begins vaccinating its citizens aged above 60 against COVID-19, with the help of Chinese-donated Sinâ€¦ https://t.co/Wn7hOtBgTb","{'id': 487118986, 'id_str': '487118986', 'name': 'China Xinhua News', 'screen_name': 'XHNews', 'location': 'Headquartered in Beijing, PRC', 'url': None, 'description': ""We are public media for the public good. We don't pursue corporate interests, nor will we yield to the pressure of ideological ...",NaN,0,2,10,45,"{'hashtags': [], 'urls': [{'url': 'https://t.co/Wn7hOtBgTb', 'expanded_url': 'https://twitter.com/i/web/status/1372079148584169481', 'display_url': 'twitter.com/i/web/status/1â€¦', 'indices': [117, 140]}], 'user_mentions': [], 'symbols': []}",live pakistan begins vaccinating its citizens aged above 60 against covid19 with the help of chinesedonated sinâ€¦
9349,Wed Mar 17 05:56:40 +0000 2021,1.370000e+18,#HeadlinesWatch: SCMP https://t.co/J15SRgQ5HM\n- 10 countries have administered 75% of all vaccinations worldwide whâ€¦ https://t.co/P6mb6xIjaZ,"{'id': 487118986, 'id_str': '487118986', 'name': 'China Xinhua News', 'screen_name': 'XHNews', 'location': 'Headquartered in Beijing, PRC', 'url': None, 'description': ""We are public media for the public good. We don't pursue corporate interests, nor will we yield to the pressure of ideological ...",NaN,5,1,16,28,"{'hashtags': [{'text': 'HeadlinesWatch', 'indices': [0, 15]}], 'urls': [{'url': 'https://t.co/J15SRgQ5HM', 'expanded_url': 'http://bit.ly/3vxPWSK', 'display_url': 'bit.ly/3vxPWSK', 'indices': [22, 45]}, {'url': 'https://t.co/P6mb6xIjaZ', 'expanded_url': 'https://twitter.com/i/web/status/13720643...",scmp 10 countries have administered 75 of all vaccinations worldwide whâ€¦


In [177]:
tweets_filtered['clean_text'].iloc[100]

'chinas home prices stay generally stable in january'

In [178]:
#initialize an empty dict
unique_words = {}
for row in tweets_filtered.clean_text:
  for word in row.split(" "):
    #if the word is encountered for the first time add to dict as key and set its value to 0
    unique_words.setdefault(word,0)
    #increase the value (i.e the count) of the word by 1 every time it is encountered
    unique_words[word] += 1

In [179]:
#remove blank space
#unique_words.pop(" ")
#remove word 'rt'
unique_words.pop("rt")

810

In [183]:
tweets_filtered = tweets_filtered.rename(columns={'favorite_count':'likes'})
ordered_likes = tweets_filtered.sort_values(by='likes',ascending = False)
pd.options.display.max_rows=5
pd.options.display.max_colwidth=300
ol = ordered_likes[['user','likes','clean_text']]
likes_10 = ol.head(10)
likes_10




,user,likes,clean_text
640,"{'id': 1115874631, 'id_str': '1115874631', 'name': 'CGTN', 'screen_name': 'CGTNOfficial', 'location': 'Beijing, China', 'url': 'http://www.CGTN.com', 'description': 'CGTN is an international media organization. It aims to provide global audiences with accurate and timely news coverage as well as...",16035,construction began on the night of january 23 for a new specialized hospital for patients in central câ€¦
2381,"{'id': 1115874631, 'id_str': '1115874631', 'name': 'CGTN', 'screen_name': 'CGTNOfficial', 'location': 'Beijing, China', 'url': 'http://www.CGTN.com', 'description': 'CGTN is an international media organization. It aims to provide global audiences with accurate and timely news coverage as well as...",16035,construction began on the night of january 23 for a new specialized hospital for patients in central câ€¦
...,...,...,...
4156,"{'id': 487118986, 'id_str': '487118986', 'name': 'China Xinhua News', 'screen_name': 'XHNews', 'location': 'Headquartered in Beijing, PRC', 'url': None, 'description': ""We are public media for the public good. We don't pursue corporate interests, nor will we yield to the pressure of ideological ...",3429,a chinese nurse in a coronavirushit hospital in henan province gives her sobbing daughter an air hugâ€¦
4335,"{'id': 487118986, 'id_str': '487118986', 'name': 'China Xinhua News', 'screen_name': 'XHNews', 'location': 'Headquartered in Beijing, PRC', 'url': None, 'description': ""We are public media for the public good. We don't pursue corporate interests, nor will we yield to the pressure of ideological ...",2900,a female doctor aiding the control in wuhan china couldnt help bursting into tears while videochattâ€¦


In [184]:
likes_10.to_csv(r"C:\Users\korab\Documents\master\network science\results\likes_10_china.csv", index=False)

In [186]:
words = pd.DataFrame.from_dict(unique_words, orient='index').reset_index()
words.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
words.sort_values(by=['Count'], ascending=False, inplace=True)
words

,Word,Count
10,the,6467
41,in,4597
...,...,...
8371,grass,1
12914,nur,1
